In [1]:
import os
os.environ['KAGGLE_USERNAME'] =
os.environ['KAGGLE_KEY'] =
!kaggle datasets download -d noorsaeed/usa-election-sentiment-analysis-dataset


Dataset URL: https://www.kaggle.com/datasets/noorsaeed/usa-election-sentiment-analysis-dataset
License(s): unknown


In [2]:
!unzip /content/usa-election-sentiment-analysis-dataset.zip

Archive:  /content/usa-election-sentiment-analysis-dataset.zip
  inflating: Bidenall2.csv           
  inflating: Trumpall2.csv           


In [3]:
import pandas as pd
df1 = pd.read_csv('/content/Bidenall2.csv')
df2 = pd.read_csv('/content/Trumpall2.csv')

In [4]:
df1.head()

,user,text
0,MarkHodder3,@JoeBiden And we’ll find out who won in 2026...
1,K87327961G,@JoeBiden Your Democratic Nazi Party cannot be...
2,OldlaceA,@JoeBiden So did Lying Barr
3,penblogger,@JoeBiden It's clear you didnt compose this tw...
4,Aquarian0264,@JoeBiden I will vote in person thank you.


In [5]:
df2.head()

,user,text
0,manny_rosen,@sanofi please tell us how many shares the Cr...
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P..."
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...
4,James09254677,After 4 years you think you would have figure...


##Preprocessing of text

In [8]:
import re
import string
import nltk
nltk.download('punkt_tab')
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

def preprocess_text(txt):
    # Lowercase
    txt = txt.lower()
    # Remove HTML tags (if any)
    txt = re.sub(r"<.*?>", " ", txt)
    # Remove URLs (if any)
    txt = re.sub(r"https?://\S+|www\.\S+", " ", txt)
    # Remove Punctuation
    txt = txt.translate(str.maketrans('', '', string.punctuation))
    # Remove numbers (optional, if you want to keep numbers, you can skip this step)
    txt = re.sub(r'[^A-Za-z\s]', '', txt)
    # Tokenize text
    words = word_tokenize(txt)

    return " ".join(words)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [9]:
df1['text'].apply(preprocess_text)

,text
0,joebiden and well find out who won in
1,joebiden your democratic nazi party can not be...
2,joebiden so did lying barr
3,joebiden its clear you didnt compose this twee...
4,joebiden i will vote in person thank you
...,...
2535,joebiden youll just try to calm those waters a...
2536,joebiden days dias votejoebiden votjoebiden
2537,joebiden you think you can do that you cant re...
2538,joebiden trump wants our children back at scho...


In [10]:
df2['text'].apply(preprocess_text)

,text
0,sanofi please tell us how many shares the crim...
1,like comment rt prayertachantitans prayertacha...
2,your ag barr is as useless amp corrupt as you ...
3,mr trump wake up most of the comments below yo...
4,after years you think you would have figured o...
...,...
2783,realdonaldtrump for the time absentee ballots ...
2784,realdonaldtrump if youre so scared of losing r...
2785,realdonaldtrump i rarely get involved with for...
2786,realdonaldtrump this is the moment when trump ...


## Find Polarity

In [12]:
from textblob import TextBlob
textblob1 = TextBlob(df1['text'][500])
textblob1.sentiment.polarity

0.6

In [14]:
from textblob import TextBlob
def find_pol(text):
  text = TextBlob(text)
  return text.sentiment.polarity

In [15]:
find_pol(df1['text'][500])

0.6

In [16]:
df1['polarity'] = df1['text'].apply(find_pol)
df2['polarity'] = df2['text'].apply(find_pol)

In [23]:
import numpy as np
# Define conditions
conditions = [
    df1['polarity'] > 0,
    df1['polarity'] < 0,
    df1['polarity'] == 0
]
# Define corresponding labels
choices = ['positive', 'negative', 'neutral']

# Apply conditions to create the 'label' column
df1['label'] = np.select(conditions, choices, default='neutral')

In [24]:
import numpy as np
# Define conditions
conditions = [
    df2['polarity'] > 0,
    df2['polarity'] < 0,
    df2['polarity'] == 0
]
# Define corresponding labels
choices = ['positive', 'negative', 'neutral']

# Apply conditions to create the 'label' column
df2['label'] = np.select(conditions, choices, default='neutral')

In [25]:
df2.head()

,user,text,polarity,label
0,manny_rosen,@sanofi please tell us how many shares the Cr...,0.05,positive
1,osi_abdul,"https://t.co/atM98CpqF7 Like, comment, RT #P...",0.00,neutral
2,Patsyrw,Your AG Barr is as useless &amp; corrupt as y...,-0.50,negative
3,seyedebrahimi_m,Mr. Trump! Wake Up! Most of the comments bel...,0.50,positive
4,James09254677,After 4 years you think you would have figure...,0.00,neutral


In [30]:
df1 = df1[df1['polarity'] != 0].reset_index(drop=True)
df2 = df2[df2['polarity'] != 0].reset_index(drop=True)

In [32]:
print(df1.shape)
print(df2.shape)

(1031, 4)
(1324, 4)


In [40]:
positive_count1 = (df1['polarity'] > 0).sum()
negative_count1 = (df1['polarity'] < 0).sum()

positive_count2 = (df2['polarity'] > 0).sum()
negative_count2 = (df2['polarity'] < 0).sum()

In [45]:
print('positive comments on Biden are: ',positive_count1)
print('negative comments on Biden are: ',negative_count1)
positive_count1 - negative_count1

positive comments on Biden are:  624
negative comments on Biden are:  407


np.int64(217)

In [44]:
print('positive comments on Trump are: ',positive_count2)
print('negative comments on Trump are: ',negative_count2)
positive_count2 - negative_count2

positive comments on Trump are:  727
negative comments on Trump are:  597


np.int64(130)